# Detector de imagenes (Fumadores)

## Importación del dataset

In [1]:
!apt-get install unrar
!pip install rarfile

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [2]:
import rarfile

rarfile.UNRAR_TOOL = "/usr/bin/unrar"
!wget https://github.com/repositoriosHackaton/SIC25es-Remember-Us-Recuerdanos-/raw/refs/heads/main/recursos/dataset.rar

rar_path = "/content/dataset.rar"  # Ruta del archivo RAR
extract_path = "dataset"  # Carpeta de salida

with rarfile.RarFile(rar_path) as rar_ref:
    rar_ref.extractall(extract_path)

print("Archivo descomprimido correctamente.")

--2025-03-25 17:07:26--  https://github.com/repositoriosHackaton/SIC25es-Remember-Us-Recuerdanos-/raw/refs/heads/main/recursos/dataset.rar
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/repositoriosHackaton/SIC25es-Remember-Us-Recuerdanos-/refs/heads/main/recursos/dataset.rar [following]
--2025-03-25 17:07:26--  https://raw.githubusercontent.com/repositoriosHackaton/SIC25es-Remember-Us-Recuerdanos-/refs/heads/main/recursos/dataset.rar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65567508 (63M) [application/octet-stream]
Saving to: ‘dataset.rar’

dataset.rar         100%[===================>]

## Entrenamiento de modelos

### SVC - Sklearn

In [3]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from PIL import Image
import numpy as np
import os

#### Tratamiento de datos

In [4]:
# Función para cargar las imagenes desde el folder y almacenarlas en forma de vector númerico, junto con otro array con su clasificación
def svc_loadImages(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename)).convert("L")
        img = img.resize((250,250))
        img_array = np.array(img).flatten()
        images.append(img_array)
        label = 0 if "notsmoking" in filename else 1 # Si el nombre de la imagen es 'notsmoking' colocar 0, caso contrario 1
        labels.append(label)
    return np.array(images), np.array(labels)

In [5]:
#Separacion imagenes train y test
scv_Xtrain, svc_ytrain = svc_loadImages("dataset/dataset/Training")
svc_Xval, svc_yval = svc_loadImages("dataset/dataset/Validation")
svc_Xtest, svc_ytest = svc_loadImages("dataset/dataset/Testing")

print(f"Datos de entrenamiento: {len(scv_Xtrain)}\tPorcentaje: {(len(scv_Xtrain)*100/1120):.2f}")
print(f"Datos de validación: {len(svc_Xval)}\tPorcentaje: {(len(svc_Xval)*100/1120):.2f}")
print(f"Datos de prueba: {len(svc_Xtest)}\t\tPorcentaje: {(len(svc_Xtest)*100/1120):.2f}")


#Escalado de imagenes con StandarScaler
scaler = StandardScaler()

scv_Xtrain_st = scaler.fit_transform(scv_Xtrain)
scv_Xval_st = scaler.transform(svc_Xval)
scv_Xtest_st = scaler.transform(svc_Xtest)

scv_Xtrain[:2], scv_Xtrain_st[:2]


#Reducir dimensionalidad con PCA a 90 componentes (componentes originales = 250)
pca = PCA(n_components=90)

scv_Xtrain_pca = pca.fit_transform(scv_Xtrain_st)
scv_Xval_pca = pca.transform(scv_Xval_st)
scv_Xtest_pca = pca.transform(scv_Xtest_st)

Datos de entrenamiento: 716	Porcentaje: 63.93
Datos de validación: 180	Porcentaje: 16.07
Datos de prueba: 224		Porcentaje: 20.00


#### Entrenamiento del modelo svc

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
import numpy as np

svm = SVC(kernel="linear", random_state=42)

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

best_score = -np.inf
best_model = None

# Realizar validación cruzada
for train_idx, val_idx in cv.split(scv_Xtrain_pca, svc_ytrain):
    X_train_fold, X_val_fold = scv_Xtrain_pca[train_idx], scv_Xtrain_pca[val_idx]
    y_train_fold, y_val_fold = svc_ytrain[train_idx], svc_ytrain[val_idx]


    svm.fit(X_train_fold, y_train_fold)

    # Evaluar el modelo
    score = svm.score(X_val_fold, y_val_fold)

    if score > best_score:
        best_score = score
        best_model = SVC(kernel="linear", random_state=42)
        best_model.fit(scv_Xtrain_pca, svc_ytrain)

# Porcenaje final
print(f"Puntuación del mejor modelo: {best_score:.2f}")

precision del 0.77

### MobileNetV2 (CNN) (keras)

In [ ]:
#algunas importacione
import os
import shutil
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Pre-procesamiento de la data

In [ ]:
#en construcion#####
# shutil.rmtree("/content/dataset/dataset/training")

ruta_origen = "dataset/dataset/Training"
ruta_smoking = "dataset/dataset/training/smoking"
ruta_notsmoking = "dataset/dataset/training/notsmoking"

os.makedirs(ruta_smoking, exist_ok=True)
os.makedirs(ruta_notsmoking, exist_ok=True)

for archivo in os.listdir(ruta_origen):
  if os.path.isfile(os.path.join(ruta_origen, archivo)):
    if "notsmoking" not in archivo.lower() :
            shutil.move(os.path.join(ruta_origen, archivo), os.path.join(ruta_smoking, archivo))
    elif(True):
            shutil.move(os.path.join(ruta_origen, archivo), os.path.join(ruta_notsmoking, archivo))


datadir = "/content/dataset/dataset/training"
imgsize = (224, 224)
batchsize = (32)

datagen = ImageDataGenerator(rescale= 1/255, validation_split=0.2)

train_data = datagen.flow_from_directory(datadir,target_size=imgsize,
                                         batch_size=batchsize,class_mode='binary',subset='training')
val_data = datagen.flow_from_directory(datadir,target_size=imgsize,
                                       batch_size=batchsize,class_mode='binary',subset='validation')

Found 574 images belonging to 2 classes.
Found 142 images belonging to 2 classes.


Instanciación del modelo (preentrenado)

In [ ]:
mnet = keras.applications.Xception(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
mnet.trainable = False

model = keras.Sequential([
    mnet,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(200, activation='relu'),
    keras.layers.Dropout(0.95),
    keras.layers.Dense(1, activation='sigmoid')  # Clasificación binaria
])
optimizer = keras.optimizers.Adam(learning_rate=1e-1)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train_data, validation_data=val_data, epochs=200)

Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - accuracy: 0.5381 - loss: 0.8822 - val_accuracy: 0.7958 - val_loss: 0.4769
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.8194 - loss: 0.4088 - val_accuracy: 0.7606 - val_loss: 0.4777
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.8725 - loss: 0.3141 - val_accuracy: 0.8099 - val_loss: 0.4155
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.8883 - loss: 0.2772 - val_accuracy: 0.8099 - val_loss: 0.3995
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.9214 - loss: 0.2218 - val_accuracy: 0.8239 - val_loss: 0.3851
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.9548 - loss: 0.1714 - val_accuracy: 0.8099 - val_loss: 0.4040
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.9484 - loss: 0.1607 - val_accuracy: 0.8239 - val_loss: 0.4025
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.9529 - loss: 0.1377 - val_accuracy: 0.8169 - val_loss:

## Yolo

In [2]:
import rarfile

rarfile.UNRAR_TOOL = "/usr/bin/unrar"
!wget https://raw.githubusercontent.com/repositoriosHackaton/SIC25es-Remember-Us-Recuerdanos-/refs/heads/main/recursos/smokersYolo.rar

rar_path = "/content/smokersYolo.rar"  # Ruta del archivo RAR
extract_path = "dataset"  # Carpeta de salida

with rarfile.RarFile(rar_path) as rar_ref:
    rar_ref.extractall(extract_path)

print("Archivo descomprimido correctamente.")

--2025-03-25 19:24:01--  https://raw.githubusercontent.com/repositoriosHackaton/SIC25es-Remember-Us-Recuerdanos-/refs/heads/main/recursos/smokersYolo.rar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5984648 (5.7M) [application/octet-stream]
Saving to: ‘smokersYolo.rar’

smokersYolo.rar     100%[===================>]   5.71M  --.-KB/s    in 0.09s   

2025-03-25 19:24:02 (62.4 MB/s) - ‘smokersYolo.rar’ saved [5984648/5984648]

Archivo descomprimido correctamente.


In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [4]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt") #tamaño nano, 90% precision

model.train(data="/content/dataset/smokersYolo/data.yaml",epochs=50,batch=8,imgsz=250,device="cuda")

model = YOLO("/content/runs/detect/train/weights/best.pt")
model.export(format="onnx")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 68.6MB/s]


Ultralytics 8.3.96 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/dataset/smokersYolo/data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=250, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True,

100%|██████████| 755k/755k [00:00<00:00, 15.2MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

                   all         60         60      0.915      0.767      0.888      0.348



50 epochs completed in 0.092 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.96 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  4.34it/s]


                   all         60         60      0.835      0.846      0.887       0.39
Speed: 0.1ms preprocess, 1.8ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/train
Ultralytics 8.3.96 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (Intel Xeon 2.30GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/content/runs/detect/train/weights/best.pt' with input shape (1, 3, 256, 256) BCHW and output shape(s) (1, 5, 1344) (5.9 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 191.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.9/142.9 kB 220.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.8/280.8 MB 170.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 123.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

'/content/runs/detect/train/weights/best.onnx'